# 3D Droplet Oscillation (Part 2, Postprocessing)

Results published: hopefully at some point!

This is part of the BoSSS-long-term validation test suite, which consists of several computationally expensive test-cases (runtime in the order of days), which are performed on a regular basis in order to validate the physical correctness of BoSSS simulations.

### Preliminaries

This example can be found in the source code repository as as `Droplet3D-Postprocessing.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [ ]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Initialization tasks

Loading the `XNSE_Solver` and additional namespace:

In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;
using BoSSS.Application.XNSE_Solver.Logging;

Initialization of the Workflow management; there `OscillatingDroplet3D` is the project name which is used name all computations (aka. sessions):

In [ ]:
BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D");

Project name is set to 'OscillatingDroplet3D'.
Opening existing database 'D:\local\OscillatingDroplet3D'.


In [ ]:
//add database manually:
OpenOrCreateDatabase(@"\\130.83.248.207\ValidationTests\OscillatingDroplet3D");

Opening existing database '\\130.83.248.207\ValidationTests\OscillatingDroplet3D'.


## Observing sessions

In [ ]:
wmg.Sessions

#0: OscillatingDroplet3D	J432k3_amr0_case1_Oh0.1_AnalytInit*	02/21/2022 12:45:02	fd6bee60...


In [ ]:
//wmg.Sessions.Where(s => s.Name.Contains("J432k3_arm0_case2_Oh0.1_AnalytInit"))
//wmg.Sessions[1].Delete(true)

Session fd6bee60-cf39-4bc0-abd9-6840a45eb932 deleted.


In [ ]:
int sNum = 0;
wmg.Sessions[sNum]

OscillatingDroplet3D	J432k3_amr0_case1_Oh0.1_AnalytInit*	02/21/2022 10:26:41	aa2d8da3...

In [ ]:
wmg.Sessions[sNum].GetSessionDirectory()

D:\local\OscillatingDroplet3D\sessions\1d6f610b-0c74-4cea-b097-7cf5f4f187d2

In [ ]:
wmg.Sessions[sNum].DeployPath

D:\local\binaries\OscillatingDroplet3D-XNSE_Solver2022Feb18_125142

In [ ]:
wmg.Sessions[sNum].Export().WithSupersampling(3).Do();

Starting export process... Data will be written to the directory: C:\Users\smuda\AppData\Local\BoSSS\plots\sessions\OscillatingDroplet3D__J432k3_amr0_case1_Oh0.1_AnalytInit__aa2d8da3-11f7-46f5-ac84-45d08bf8685d


In [ ]:
using System.IO;

In [8]:
/*
// temporary fix for a bug in the output file "SphericalHarmonics.txt"
// (missing separator/tab between fist ans second column)
void FileSanitizer(string _TextFile) {
Console.WriteLine("sanitizing: " + _TextFile);
var TextFile2 = Path.Combine(Path.GetDirectoryName(_TextFile), Path.GetFileNameWithoutExtension(_TextFile) + "-Copy.txt");
File.Copy(_TextFile,TextFile2, true);
var TextFile3 = Path.Combine(Path.GetDirectoryName(_TextFile), Path.GetFileNameWithoutExtension(_TextFile) + "-Sanitized.txt");
using (StreamWriter wrt = new StreamWriter(TextFile3)) {
using (StreamReader reader = new StreamReader(new FileStream(TextFile2, FileMode.Open, FileAccess.Read, FileShare.Read))) {
  int cnt = 0;
  for(string l = reader.ReadLine(); l != null; l = reader.ReadLine()) {
     if(cnt >= 1) {
       string[] parts = l.Split("\t");
       //Console.WriteLine(parts[0] + "    " + parts.Length);
       string P0 = parts[0];
       int L = P0.Length;
       
       string pp1, pp2;
       int exp = P0.IndexOf("E", 0, 5);
       //exp = -1;
       if(exp >= 0) {
          int ppl = P0[exp + 1] == '-' ? 4 : 3;
          pp1 = P0.Substring(0,exp + ppl);
          pp2 = P0.Substring(exp + ppl);
          
       } else {
          int minus = P0.IndexOf("-");
          if(minus >= 0 && P0[minus - 1] == 'E')
             minus = -1;
          if(minus >= 0) {
              pp1 = P0.Substring(0,minus);
              pp2 = P0.Substring(minus);
          } else {
              int comma = P0.IndexOf('.', 2, L - 2);
              pp1 = P0.Substring(0,comma - 1);
              pp2 = P0.Substring(comma - 1);
          }
       }
       try {
          double.Parse(pp1);
       } catch(Exception) {
          Console.Error.WriteLine("line " + (cnt + 1) + " cannot parse 1: " + pp1 + "  from " + P0);
       }
       try {
            double.Parse(pp2);
       } catch(Exception) {
          Console.Error.WriteLine("line " + (cnt + 1) + " cannot parse 2: " + pp2 + "  from " + P0);
       }
       wrt.Write(pp1 + "\t" + pp2);

       
       for(int i = 1; i < parts.Length; i++) {
            wrt.Write("\t");
            wrt.Write(parts[i]);
       }
       wrt.WriteLine();
       
     } else {
       wrt.WriteLine(l);
     }
     cnt++;
  }
}
}

}
*/

In [9]:
/*
foreach(var s in wmg.Sessions) {
    string dir = DatabaseDriver.GetSessionDirectory(s);
    string file = Path.Combine(dir, "SphericalHarmonics.txt");
    FileSanitizer(file);
}
*/

In [10]:
// might take some time
/*
foreach(var s in wmg.Sessions) {
    Console.Write(s.Name + ": ");
    if(s.Timesteps.Count() > 0)
        Console.WriteLine(s.Timesteps.Last());
    else 
        Console.WriteLine("no timestep computed/saved");
    Console.WriteLine();
}
*/

In [11]:
//wmg.Sessions.Single(si => si.Name.Contains("case3")).Export().WithSupersampling(2).Do()

## Plotting of Spherical Harmonics

In [ ]:
//string[] modes = new string[]{ "mode2", "mode3", "mode4"};
//string[] aPis = new string[] { "aP0", "aP1", "aP2" };
string[] amrS = new string[] { "amr1" };
string[] caseS = new string[] { "case1", "case2", "case3", "case4", "case5" };
bool[] ShouldAnaInit = new bool[] { true };
bool[] useNewton = new bool[] { true, false };
bool skipNonPoly = true; // false: plots all Legendgre functions, i.e. indices (*, *)
//                          true:  plots only Legendre polynomials, i.e. inices (*, 0)

In [ ]:
Plot2Ddata[,] PlotTableSH = new Plot2Ddata[5,1];
for(int iCol = 0; iCol < 1; iCol++) {
for(int iRow = 0; iRow < 5; iRow++) { // loop over cases
for(int iAMR = 0; iAMR < 1; iAMR++) {
for(int iAna = 0; iAna < 1; iAna++) {
for(int iNew = 0; iNew < 2; iNew++) {
    string _amr = amrS[iAMR];
    string _case = caseS[iRow];

    //ISessionInfo SI = null;
    var SIs = wmg.Sessions.Where(sess => sess.Name.Contains(_case) 
                                      && sess.Name.Contains(_amr)
                                      && (sess.Name.Contains("_AnalytInit") == ShouldAnaInit[iAna])
                                      && (sess.Name.Contains("_Newton") == useNewton[iNew]));
    
    if(SIs.Count() > 1) {
        foreach(var s in SIs)
            Console.WriteLine(s);
    }
    
    ISessionInfo SI = SIs.Single();
    if(SI == null)
        continue;
    Console.WriteLine(SI.Name);

    IDictionary<string, IList<double>> tab = null;
    try {
        tab = SI.ReadTabulatedTextFileAsDoubles("SphericalHarmonics.txt", '\t');
    } catch (Exception e) {
        Console.Error.WriteLine(e.Message);
        Console.WriteLine("Skipping " + SI);
        
        var dummyPlot = new Plot2Ddata();
        dummyPlot.AddDataGroup("empty", new double[]{ 0, 1 }, new double[] { 0, 1 });
        PlotTableSH[iRow,iCol] = dummyPlot;
        continue;
    }
    
    var plot = new Plot2Ddata();
    var allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
    var time = tab["time"];
    int cnt = -1;
    foreach(var column in tab) {
       cnt++;
       var fmt = new PlotFormat();
       fmt.Style = Styles.Lines; 
       fmt.LineColor = allColors[cnt%allColors.Length];
       if(column.Key == "time")
          continue;
       if(skipNonPoly) {
           if(!column.Key.Contains(", 0)")) {
               continue;
           }
       } 
        
        // if(iAMR == 1 && ShouldAnaInit[iAna] == false)
        //     fmt.DashType = DashTypes.Solid;
        // else if(iAMR == 1 && ShouldAnaInit[iAna] == true)
        //     fmt.DashType = DashTypes.Dashed;
        // else if(iAMR == 0 && ShouldAnaInit[iAna] == false)
        //     fmt.DashType = DashTypes.Dotted;
        // else if(iAMR == 0 && ShouldAnaInit[iAna] == true)
        //     fmt.DashType = DashTypes.DotDotDashed;
        // else
        //     throw new Exception("unknown setting");
        if (useNewton[iNew]) 
            fmt.DashType = DashTypes.Dashed;
        else  
            fmt.DashType = DashTypes.Solid;

        string name = column.Key;
        if(name.Contains(", 0)"))
            fmt.LineWidth = 3;
        if(iAMR > 0)
            name = name + "-amr";
        if(ShouldAnaInit[iAna])
            name = name + "-Ainit";
        if(useNewton[iNew])
            name = name + "-Newton";
        
        plot.AddDataGroup(name, time, column.Value, fmt);
   }
   
   //plot.ShowLegend = iCol == 2 && iRow == 2;
   plot.ShowLegend = true;
   
   //plot.Title = SI.Name;
    
   if(iAMR == 0) {
       PlotTableSH[iRow,iCol] = plot;
   } else {
       PlotTableSH[iRow,iCol] = PlotTableSH[iRow,iCol].Merge(plot);
   }
   
    
   PlotTableSH[iRow,iCol].XrangeMin = 0;
   PlotTableSH[iRow,iCol].XrangeMax = 4;
}

if(PlotTableSH[iRow,iCol] != null) {
    PlotTableSH[iRow,iCol].ShowLegend = true; //iCol == 0 && iRow == 0;
    PlotTableSH[iRow,iCol].LegendFont = 10;
    //if(iRow == 0)
    //   PlotTable[iRow,iCol].Title = aPis[iCol];
    if(iCol == 0)
       PlotTableSH[iRow,iCol].Ylabel = caseS[iRow];
}
}
}
}
}

J432k3_amr1_case1_Oh0.1_AnalytInit_Newton
J432k3_amr1_case1_Oh0.1_AnalytInit
J432k3_amr1_case2_Oh0.1_AnalytInit_Newton
J432k3_amr1_case2_Oh0.1_AnalytInit
J432k3_amr1_case3_Oh0.1_AnalytInit_Newton
J432k3_amr1_case3_Oh0.1_AnalytInit
J432k3_amr1_case4_Oh0.1_AnalytInit_Newton
J432k3_amr1_case4_Oh0.1_AnalytInit
J432k3_amr1_case5_Oh0.56_AnalytInit_Newton
J432k3_amr1_case5_Oh0.56_AnalytInit


### Case 1

In [ ]:
PlotTableSH[0,0].XrangeMin = 0;
PlotTableSH[0,0].XrangeMax = 1;
PlotTableSH[0,0].Legend_maxrows = 4;
PlotTableSH[0,0].LegendAlignment = new [] { "i", "t", "c" };
var gp = PlotTableSH[0,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",10" inside top center  maxrows 4 Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.99 
 
 
 
 
 0.995 
 
 
 
 
 1 
 
 
 
 
 1.005 
 
 
 
 
 1.01 
 
 
 
 
 1.015 
 
 
 
 
 1.02 
 
 
 
 
 1.025 
 
 
 
 
 1.03 
 
 
 
 
 1.035 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 case1 
 
 
 
 
 (0, 0)-Ainit 
 
 
 
 
 (0, 0)-Ainit 
 
 
 
	<path stroke='rgb( 0, 255, 0)' d='M359.6,52.6 L396.2,52.6 M88.5,51.8 L91.8,170.6 L95.1,176.6 L98.4,180.1 L101.7,185.4 L105.0,190.2
 L108.3,195.0 L111.7,199.9 L115.0,204.8 L118.3,209.9 L121.6,215.0 L124.9,220.2 L128.2,225.4 L131.5,230.7
 L134.8,236.1 L138.1,241.4 L141.4,246.9 L144.7,252.3 L148.0,257.8 L151.4,263.2 L154.7,268.7 L158.0,274.2
 L161.3,279.7 L164.6,285.1 L167.9,290.6 L171.2,296.0 L174.5,301.3 L177.8,306.7 L181.1,312.0 L184.4,317.2
 L187.7,322.4 L191.0,327.6 L194.4,332.8 L197.7,337.9 L201.0,342.9 L204.3,347.9 L207.6,352.9 L210.9,357.9
 L214.2,362.8 L217.5,367.7 L220.8,372.6 L224.1,377.4 L227.4,382.2 L230.7,387.0 L234.1,391.8 L237.4,396.5
 L240.7,401.1 L244.0,405.7 L247.3,410.3 L250.6,414.8 L253.9,419.3 L257.2,423.7 L260.5,428.1 L263.8,432.4
 L267.1,436.6 L270.4,440.7 L273.7,444.8 L277.1,448.8 L280.4,452.7 L283.7,456.5 L287.0,460.1 L290.3,463.8
 L293.6,466.8 L296.9,470.2 L300.2,473.6 L303.5,476.8 L306.8,480.0 L310.1,483.1 L313.4,486.1 L316.8,488.9
 L320.1,491.6 L323.4,494.2 L326.7,496.7 L330.0,499.0 L333.3,501.3 L336.6,503.4 L339.9,505.4 L343.2,507.3
 L346.5,509.0 L349.8,510.7 L353.1,512.2 L356.4,513.7 L359.8,515.0 L363.1,516.2 L366.4,517.3 L369.7,518.3
 L373.0,519.2 L376.3,520.0 L379.6,520.7 L382.9,521.3 L386.2,521.8 L389.5,524.0 L392.8,523.3 L396.1,523.3
 L399.5,523.3 L402.8,523.4 L406.1,523.5 L409.4,523.4 L412.7,523.3 L416.0,523.0 L419.3,522.8 L422.6,522.5
 L425.9,522.1 L429.2,521.6 L432.5,521.0 L435.8,520.4 L439.1,519.8 L442.5,519.0 L445.8,518.2 L449.1,517.4
 L452.4,516.5 L455.7,515.5 L459.0,514.6 L462.3,513.6 L465.6,512.5 L468.9,511.4 L472.2,510.3 L475.5,509.2
 L478.8,509.5 L482.2,507.4 L485.5,505.9 L488.8,504.3 L492.1,502.9 L495.4,501.4 L498.7,500.0 L502.0,498.5
 L505.3,497.0 L508.6,495.5 L511.9,494.0 L515.2,492.5 L518.5,491.0 L521.8,489.4 L525.2,487.8 L528.5,486.2
 L531.8,484.6 L535.1,483.0 L538.4,481.3 L541.7,479.7 L545.0,478.0 L548.3,476.4 L551.6,474.7 L554.9,473.0
 L558.2,471.4 L561.5,469.7 L564.9,468.0 L568.2,466.1 L571.5,464.5 L574.8,462.9 L578.1,461.3 L581.4,459.7
 L584.7,458.2 L588.0,456.6 L591.3,455.1 L594.6,453.6 L597.9,452.2 L601.2,450.7 L604.5,449.3 L607.9,447.9
 L611.2,446.6 L614.5,445.2 L617.8,443.9 L621.1,442.7 L624.4,441.4 L627.7,440.2 L631.0,439.1 L634.3,438.0
 L637.6,436.9 L640.9,435.8 L644.2,434.8 L647.6,433.8 L650.9,432.9 L654.2,432.0 L657.5,431.2 '/>

### Case 2

In [ ]:
PlotTableSH[1,0].XrangeMin = 0;
PlotTableSH[1,0].XrangeMax = 4;
PlotTableSH[1,0].Legend_maxrows = 4;
PlotTableSH[1,0].LegendAlignment = new [] { "i", "c", "c" };
var gp = PlotTableSH[1,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",10" inside center center  maxrows 4 Left reverse 
Gnuplot Error: Warning: empty y range [1.0154:1.0154], adjusting to [1.00524:1.02555]



<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.005 
 
 
 
 
 1.01 
 
 
 
 
 1.015 
 
 
 
 
 1.02 
 
 
 
 
 1.025 
 
 
 
 
 1.03 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 case2 
 
 
 
 
 (0, 0)-Ainit 
 
 
 
 
 (0, 0)-Ainit

### Case 3

In [ ]:
PlotTableSH[2,0].XrangeMin = 0;
PlotTableSH[2,0].XrangeMax = 4;
PlotTableSH[2,0].Legend_maxrows = 4;
PlotTableSH[2,0].LegendAlignment = new [] { "i", "t", "c" };
var gp = PlotTableSH[2,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",10" inside top center  maxrows 4 Left reverse 
Gnuplot Error: Warning: empty y range [1.01717:1.01717], adjusting to [1.007:1.02734]



<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.005 
 
 
 
 
 1.01 
 
 
 
 
 1.015 
 
 
 
 
 1.02 
 
 
 
 
 1.025 
 
 
 
 
 1.03 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 case3 
 
 
 
 
 (0, 0)-Ainit 
 
 
 
 
 (0, 0)-Ainit

### Case 4

In [ ]:
PlotTableSH[3,0].XrangeMin = 0;
PlotTableSH[3,0].XrangeMax = 1;
PlotTableSH[3,0].Legend_maxrows = 4;
PlotTableSH[3,0].LegendAlignment = new [] { "i", "c", "c" };
var gp = PlotTableSH[3,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",10" inside center center  maxrows 4 Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.999 
 
 
 
 
 1 
 
 
 
 
 1.001 
 
 
 
 
 1.002 
 
 
 
 
 1.003 
 
 
 
 
 1.004 
 
 
 
 
 1.005 
 
 
 
 
 1.006 
 
 
 
 
 1.007 
 
 
 
 
 1.008 
 
 
 
 
 1.009 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 case4 
 
 
 
 
 (0, 0)-Ainit 
 
 
 
 
 (0, 0)-Ainit 
 
 
 
	<path stroke='rgb( 0, 255, 0)' d='M359.6,300.1 L396.2,300.1 M88.5,63.1 L91.8,67.8 L95.1,67.6 L98.4,67.8 L101.7,68.3 L105.0,68.9
 L108.3,69.8 L111.7,70.8 L115.0,71.9 L118.3,73.2 L121.6,74.6 L124.9,76.1 L128.2,77.7 L131.5,79.5
 L134.8,81.4 L138.1,83.4 L141.4,85.5 L144.7,87.7 L148.0,90.0 L151.4,92.4 L154.7,95.0 L158.0,97.6
 L161.3,100.4 L164.6,103.3 L167.9,106.2 L171.2,109.3 L174.5,112.4 L177.8,115.7 L181.1,119.0 L184.4,122.5
 L187.7,126.0 L191.0,129.6 L194.4,133.3 L197.7,137.1 L201.0,141.0 L204.3,144.9 L207.6,149.0 L210.9,153.1
 L214.2,157.2 L217.5,161.5 L220.8,165.8 L224.1,170.2 L227.4,174.7 L230.7,179.2 L234.1,183.7 L237.4,188.3
 L240.7,194.6 L244.0,199.4 L247.3,204.6 L250.6,208.6 L253.9,212.9 L257.2,217.5 L260.5,222.2 L263.8,227.1
 L267.1,232.1 L270.4,237.1 L273.7,242.2 L277.1,247.2 L280.4,252.3 L283.7,257.4 L287.0,262.6 L290.3,267.7
 L293.6,272.8 L296.9,278.0 L300.2,283.2 L303.5,288.3 L306.8,293.5 L310.1,298.6 L313.4,303.7 L316.8,308.8
 L320.1,314.0 L323.4,319.0 L326.7,324.1 L330.0,329.2 L333.3,334.2 L336.6,339.2 L339.9,344.2 L343.2,349.2
 L346.5,354.1 L349.8,358.9 L353.1,363.8 L356.4,368.6 L359.8,373.4 L363.1,378.1 L366.4,382.7 L369.7,387.4
 L373.0,391.9 L376.3,396.4 L379.6,400.9 L382.9,405.3 L386.2,409.6 L389.5,413.9 L392.8,418.1 L396.1,422.2
 L399.5,426.3 L402.8,430.3 L406.1,434.2 L409.4,438.0 L412.7,441.8 L416.0,445.5 L419.3,449.1 L422.6,452.6
 L425.9,456.0 L429.2,459.4 L432.5,462.7 L435.8,465.8 L439.1,468.9 L442.5,471.9 L445.8,474.8 L449.1,477.6
 L452.4,480.3 L455.7,483.0 L459.0,485.6 L462.3,488.0 L465.6,490.4 L468.9,492.7 L472.2,494.8 L475.5,496.9
 L478.8,498.9 L482.2,500.8 L485.5,502.5 L488.8,504.2 L492.1,505.8 L495.4,507.2 L498.7,508.6 L502.0,509.9
 L505.3,511.0 L508.6,512.1 L511.9,513.1 L515.2,513.9 L518.5,514.7 L521.8,518.9 L525.2,519.2 L528.5,519.9
 L531.8,518.9 L535.1,518.4 L538.4,518.2 L541.7,518.0 L545.0,517.9 L548.3,517.8 L551.6,517.6 L554.9,517.3
 L558.2,516.9 L561.5,516.5 L564.9,519.0 L568.2,517.1 L571.5,515.6 L574.8,514.5 L578.1,513.5 L581.4,512.5
 L584.7,511.5 L588.0,510.5 L591.3,509.3 L594.6,508.2 L597.9,506.9 L601.2,505.7 L604.5,504.3 L607.9,503.0
 L611.2,501.5 L614.5,500.0 L617.8,498.5 L621.1,496.9 L624.4,495.3 L627.7,493.6 L631.0,491.9 L634.3,490.1
 L637.6,488.3 L640.9,486.3 L644.2,484.4 L647.6,482.4 L650.9,480.5 L654.2,478.6 L657.5,476.6 L660.8,474.6
 L664.1,472.6 L667.4,470.6 L670.7,468.5 L674.0,466.5 L677.3,464.4 L680.6,462.3 L683.9,460.2 L687.2,458.1
 L690.6,456.1 L693.9,454.0 L697.2,451.9 L700.5,449.8 L703.8,447.7 L707.1,445.7 L710.4,443.6 L713.7,441.5
 L717.0,439.5 L720.3,437.5 L723.6,435.5 L726.9,433.5 L730.3,431.5 L733.6,429.6 L736.9,427.7 '/>

### Case 5

In [ ]:
PlotTableSH[4,0].XrangeMin = 0;
PlotTableSH[4,0].XrangeMax = 2;
PlotTableSH[4,0].Legend_maxrows = 4;
PlotTableSH[4,0].LegendAlignment = new [] { "i", "c", "c" };
var gp = PlotTableSH[4,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",10" inside center center  maxrows 4 Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.9986 
 
 
 
 
 0.9988 
 
 
 
 
 0.999 
 
 
 
 
 0.9992 
 
 
 
 
 0.9994 
 
 
 
 
 0.9996 
 
 
 
 
 0.9998 
 
 
 
 
 1 
 
 
 
 
 1.0002 
 
 
 
 
 1.0004 
 
 
 
 
 1.0006 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 case5 
 
 
 
 
 (0, 0)-Ainit 
 
 
 
 
 (0, 0)-Ainit 
 
 
 
	<path stroke='rgb( 0, 255, 0)' d='M363.7,300.1 L400.3,300.1 M96.8,87.8 L98.4,462.2 L100.1,450.6 L101.7,440.9 L103.3,432.4 L105.0,424.6
 L106.6,417.6 L108.2,411.5 L109.9,406.1 L111.5,401.6 L113.1,397.7 L114.8,394.6 L116.4,392.1 L118.0,390.3
 L119.7,389.0 L121.3,388.3 L122.9,388.0 L124.6,388.3 L126.2,388.9 L127.8,390.0 L129.5,391.4 L131.1,393.1
 L132.7,395.0 L134.4,397.2 L136.0,399.6 L137.6,402.2 L139.3,404.9 L140.9,407.8 L142.5,410.8 L144.2,413.9
 L145.8,417.0 L147.4,420.2 L149.1,423.4 L150.7,426.6 L152.3,429.9 L154.0,433.1 L155.6,436.4 L157.2,439.6
 L158.9,442.7 L160.5,445.9 L162.1,449.0 L163.8,452.0 L165.4,455.0 L167.0,458.0 L168.7,460.8 L170.3,463.6
 L171.9,466.4 L173.6,469.1 L175.2,471.7 L176.8,474.2 L178.5,476.7 L180.1,479.1 L181.7,481.4 L183.4,483.7
 L185.0,485.9 L186.6,488.0 L188.3,490.1 L189.9,492.1 L191.5,494.0 L193.2,495.8 L194.8,497.6 L196.4,499.4
 L198.1,501.1 L199.7,502.7 L201.3,504.2 L203.0,505.7 L204.6,507.2 L206.2,508.6 L207.9,509.9 L209.5,511.2
 L211.1,512.6 L212.8,513.9 L214.4,515.0 L216.0,516.1 L217.7,517.2 L219.3,518.2 L220.9,519.2 L222.6,520.2
 L224.2,521.1 L225.8,522.3 L227.5,523.1 L229.1,523.9 L230.7,524.7 L232.4,525.5 L234.0,526.2 L235.6,527.0
 L237.3,527.6 L238.9,528.3 L240.5,528.9 L242.2,529.5 L243.8,530.1 L245.4,530.7 L247.1,531.2 L248.7,531.7
 L250.3,532.2 L252.0,532.7 L253.6,533.1 L255.2,533.6 L256.9,534.0 L258.5,534.4 L260.1,534.8 L261.8,535.2
 L263.4,535.5 L265.0,535.8 L266.7,536.2 L268.3,536.5 L269.9,536.8 L271.6,537.1 L273.2,537.4 L274.8,537.6
 L276.5,537.9 L278.1,538.2 L279.7,538.4 L281.4,538.6 L283.0,538.8 L284.6,539.1 L286.3,539.3 L287.9,539.4
 L289.5,539.6 L291.2,539.8 L292.8,540.0 L294.4,540.2 L296.1,540.3 L297.7,540.5 L299.3,540.6 L301.0,540.7
 L302.6,540.9 L304.2,541.0 L305.9,541.1 L307.5,541.3 L309.1,541.4 L310.8,541.5 L312.4,541.6 L314.0,541.7
 L315.7,541.8 L317.3,541.9 L318.9,542.0 L320.6,542.0 L322.2,542.1 L323.8,542.2 L325.5,542.3 L327.1,542.4
 L328.7,542.4 L330.4,542.5 L332.0,542.6 L333.6,542.6 L335.3,542.7 L336.9,542.7 L338.5,542.8 L340.2,542.8
 L341.8,542.9 L343.4,542.9 L345.1,543.0 L346.7,543.0 L348.3,543.1 L350.0,543.1 L351.6,543.2 L353.2,543.2
 L354.9,543.2 L356.5,543.3 L358.1,543.3 L359.8,543.3 L361.4,543.4 L363.0,543.4 L364.7,543.4 L366.3,543.4
 L367.9,543.5 L369.6,543.5 L371.2,543.5 L372.8,543.5 L374.5,543.6 L376.1,543.6 L377.7,543.6 L379.4,543.6
 L381.0,543.6 L382.6,543.7 L384.3,543.7 L385.9,543.7 L387.5,543.7 L389.2,543.7 L390.8,543.7 L392.4,543.7
 L394.1,543.8 L395.7,543.8 L397.3,543.8 L399.0,543.8 L400.6,543.8 L402.2,543.8 L403.9,543.8 L405.5,543.8
 L407.1,543.8 L408.8,543.9 L410.4,543.9 L412.0,543.9 L413.7,543.9 L415.3,543.9 L416.9,543.9 L418.6,543.9
 L420.2,543.9 L421.8,543.9 L423.5,543.9 L425.1,543.9 L426.7,543.9 L428.3,543.9 L430.0,544.0 L431.6,544.0
 L433.2,544.0 L434.9,544.0 L436.5,544.0 L438.1,544.0 L439.8,544.0 L441.4,544.0 L443.0,544.0 L444.7,544.0
 L446.3,544.0 L447.9,544.0 L449.6,544.1 L451.2,544.0 L452.8,544.0 L454.5,544.0 L456.1,543.5 L457.7,544.1
 L459.4,544.1 L461.0,544.1 L462.6,544.1 L464.3,544.1 L465.9,544.0 L467.5,544.1 L469.2,544.0 L470.8,544.0
 L472.4,544.0 L474.1,544.0 '/>

### All-in-one plot

In [48]:
for(int i = 0; i < PlotTableSH.GetLength(0); i++) {
   PlotTableSH[i,0].XrangeMin = 0;
   PlotTableSH[i,0].XrangeMax = 2.5;
}

In [49]:
var gp = PlotTableSH.ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",10" inside top center  maxrows 4 Left reverse 
set key font ",10" inside center center  maxrows 4 Left reverse 
set key font ",10" inside top center  maxrows 4 Left reverse 
set key font ",10" inside center center  maxrows 4 Left reverse 
set key font ",10" inside center center  maxrows 4 Left reverse 



warning CS1701: Assuming assembly reference 'Microsoft.AspNetCore.Html.Abstractions, Version=2.2.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' used by 'BoSSSpad' matches identity 'Microsoft.AspNetCore.Html.Abstractions, Version=5.0.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' of 'Microsoft.AspNetCore.Html.Abstractions', you may need to supply runtime policy



<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.4 
 
 
 
 
 -0.2 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 1.4 
 
 
 
 
 1.6 
 
 
 
 
 1.8 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 case1 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0) 
 
 
 
	<path stroke='rgb( 0, 255, 0)' stroke-dasharray='3.0,6.0' d='M676.6,55.6 L713.2,55.6 M89.8,124.6 L93.1,124.3 L96.4,124.2 L99.7,124.2 L102.9,124.3 L106.2,124.5
 L109.5,124.7 L112.8,125.0 L116.1,125.3 L119.4,125.7 L122.6,126.2 L125.9,126.7 L129.2,127.2 L132.5,127.8
 L135.8,128.4 L139.1,129.1 L142.4,129.8 L145.6,130.5 L148.9,131.3 L152.2,132.0 L155.5,132.8 L158.8,133.6
 L162.1,134.5 L165.3,135.3 L168.6,136.2 L171.9,137.0 L175.2,137.9 L178.5,138.7 L181.8,139.6 L185.1,140.4
 L188.3,141.2 L191.6,142.1 L194.9,142.9 L198.2,143.6 L201.5,144.4 L204.8,145.2 L208.0,145.9 L211.3,146.6
 L214.6,147.2 L217.9,147.9 L221.2,148.5 L224.5,149.1 L227.8,149.6 L231.0,150.1 L234.3,150.6 L237.6,151.1
 L240.9,151.5 L244.2,151.8 L247.5,152.2 L250.7,152.5 L254.0,152.7 L257.3,153.0 L260.6,153.2 L263.9,153.3
 L267.2,153.4 L270.5,153.5 L273.7,153.6 L277.0,153.6 L280.3,153.6 L283.6,153.5 L286.9,153.5 L290.2,153.4
 L293.4,153.2 L296.7,153.1 L300.0,152.9 L303.3,152.7 L306.6,152.5 L309.9,152.2 L313.2,151.9 L316.4,151.6
 L319.7,151.3 L323.0,151.0 L326.3,150.7 L329.6,150.3 L332.9,150.0 L336.1,149.6 L339.4,149.2 L342.7,148.8
 L346.0,148.4 L349.3,148.0 L352.6,147.6 L355.9,147.2 L359.1,146.7 L362.4,146.3 L365.7,145.9 L369.0,145.5
 L372.3,145.1 L375.6,144.6 L378.8,144.2 L382.1,143.8 L385.4,143.4 L388.7,143.0 L392.0,142.6 L395.3,142.2
 L398.6,141.8 L401.8,141.4 L405.1,141.0 L408.4,140.6 L411.7,140.2 L415.0,139.9 L418.3,139.5 L421.5,139.1
 L424.8,138.7 L428.1,138.4 L431.4,138.0 L434.7,137.7 L438.0,137.3 L441.3,137.0 L444.5,136.7 L447.8,136.3
 L451.1,136.0 L454.4,135.7 L457.7,135.4 L461.0,135.1 L464.2,134.8 L467.5,134.5 L470.8,135.1 L474.1,134.2
 L477.4,133.7 L480.7,133.3 L484.0,133.0 L487.2,132.7 L490.5,132.4 L493.8,132.1 L497.1,131.8 L500.4,131.5
 L503.7,131.2 L506.9,130.9 L510.2,130.6 L513.5,130.3 L516.8,130.0 L520.1,129.7 L523.4,129.4 L526.7,129.1
 L529.9,128.8 L533.2,128.5 L536.5,128.2 L539.8,127.9 L543.1,127.6 L546.4,127.3 L549.6,127.0 L552.9,126.7
 L556.2,126.4 L559.5,126.1 L562.8,125.9 L566.1,125.6 L569.4,125.4 L572.6,125.1 L575.9,124.9 L579.2,124.6
 L582.5,124.4 L585.8,124.2 L589.1,124.0 L592.3,124.5 L595.6,124.1 L598.9,124.2 L602.2,123.7 L605.5,123.4
 L608.8,123.2 L612.1,122.9 L615.3,122.7 L618.6,122.4 L621.9,122.1 L625.2,121.8 L628.5,121.4 L631.8,121.6
 L635.0,120.9 L638.3,120.5 L641.6,120.0 L644.9,119.7 L648.2,119.3 L651.5,119.0 L654.8,118.8 L658.0,118.5
 L661.3,118.3 L664.6,118.2 L667.9,116.1 L671.2,114.6 L674.5,113.4 L677.7,112.5 L681.0,111.9 L684.3,111.4
 L687.6,111.1 L690.9,110.9 L694.2,110.9 L697.5,111.0 L700.7,111.2 L704.0,111.5 L707.3,111.8 L710.6,112.2
 L713.9,112.7 L717.2,113.2 L720.4,113.7 L723.7,114.2 L727.0,114.8 L730.3,115.3 L733.6,115.9 L736.9,116.5
 L740.2,117.1 L743.4,117.7 L746.7,118.3 L750.0,118.8 L753.3,119.4 L756.6,120.0 L759.9,120.6 L763.1,121.1
 L766.4,121.7 L769.7,122.2 L773.0,122.8 L776.3,123.3 L779.6,123.8 L782.9,124.3 L786.1,124.8 L789.4,125.3
 L792.7,125.8 L796.0,126.2 L799.3,126.7 L802.6,127.1 L805.8,127.6 L809.1,128.0 L812.4,128.4 L815.7,128.8
 L819.0,129.1 L822.3,129.5 L825.6,129.8 L828.8,130.2 L832.1,130.5 L835.4,130.8 L838.7,131.1 L842.0,131.4
 L845.3,131.6 L848.5,131.9 L851.8,132.1 L855.1,132.3 L858.4,132.6 L861.7,132.7 L865.0,132.9 L868.3,133.1
 L871.5,133.2 L874.8,133.4 L878.1,133.5 L

In [ ]:
//Note: you must create sub-directory "latex" first!
//var cl = gp.PlotCairolatex(xSize:32,ySize:18);
//cl.WriteMinimalCompileableExample("latex\\plot.tex");

In [ ]:
foreach(var S in wmg.Sessions) {
    //var EI = S.Export().WithSupersampling(1).Do();
}

## Plotting  droplet metrics

In [ ]:
string[] metricKeys = new string[] {"theta0", "theta90x", "theta90y", "volume"};

In [ ]:
Plot2Ddata[,] PlotTableDM = new Plot2Ddata[5,1];
for(int iCol = 0; iCol < 1; iCol++) {
for(int iRow = 0; iRow < 5; iRow++) { // loop over cases
for(int iAMR = 0; iAMR < 1; iAMR++) {
for(int iAna = 0; iAna < 1; iAna++) {
for(int iNew = 0; iNew < 2; iNew++) {
    string _amr = amrS[iAMR];
    string _case = caseS[iRow];

    //ISessionInfo SI = null;
    var SIs = wmg.Sessions.Where(sess => sess.Name.Contains(_case) 
                                      && sess.Name.Contains(_amr)
                                      && (sess.Name.Contains("_AnalytInit") == ShouldAnaInit[iAna])
                                      && (sess.Name.Contains("_Newton") == useNewton[iNew]));
    
    if(SIs.Count() > 1) {
        foreach(var s in SIs)
            Console.WriteLine(s);
    }
    
    ISessionInfo SI = SIs.Single();
    if(SI == null)
        continue;
    Console.WriteLine(SI.Name);

    IDictionary<string, IList<double>> tab = null;
    try {
        tab = SI.ReadTabulatedTextFileAsDoubles("DropletMetrics.txt", '\t');
    } catch (Exception e) {
        Console.Error.WriteLine(e.Message);
        Console.WriteLine("Skipping " + SI);
        
        var dummyPlot = new Plot2Ddata();
        dummyPlot.AddDataGroup("empty", new double[]{ 0, 1 }, new double[] { 0, 1 });
        PlotTableDM[iRow,iCol] = dummyPlot;
        continue;
    }
    
    var plot = new Plot2Ddata();
    var allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
    var time = tab["time"];
    int cnt = -1;
    foreach(var key in metricKeys) {
       cnt++;
       var fmt = new PlotFormat();
       fmt.Style = Styles.Lines; 
       fmt.LineColor = allColors[cnt%allColors.Length];
        
        // if(iAMR == 1 && ShouldAnaInit[iAna] == false)
        //     fmt.DashType = DashTypes.Solid;
        // else if(iAMR == 1 && ShouldAnaInit[iAna] == true)
        //     fmt.DashType = DashTypes.Dashed;
        // else if(iAMR == 0 && ShouldAnaInit[iAna] == false)
        //     fmt.DashType = DashTypes.Dotted;
        // else if(iAMR == 0 && ShouldAnaInit[iAna] == true)
        //     fmt.DashType = DashTypes.DotDotDashed;
        // else
        //     throw new Exception("unknown setting");
        if (useNewton[iNew]) 
            fmt.DashType = DashTypes.Dashed;
        else  
            fmt.DashType = DashTypes.Solid;

        string name = key;
        if(iAMR > 0)
            name = name + "-amr";
        if(ShouldAnaInit[iAna])
            name = name + "-Ainit";
        if(useNewton[iNew])
            name = name + "-Newton";
        
        plot.AddDataGroup(name, time, tab[key], fmt);
   }
   
   //plot.ShowLegend = iCol == 2 && iRow == 2;
   plot.ShowLegend = true;
   
   //plot.Title = SI.Name;
    
   if(iAMR == 0) {
       PlotTableDM[iRow,iCol] = plot;
   } else {
       PlotTableDM[iRow,iCol] = PlotTableDM[iRow,iCol].Merge(plot);
   }
   
    
   PlotTableDM[iRow,iCol].XrangeMin = 0;
   PlotTableDM[iRow,iCol].XrangeMax = 4;
}

if(PlotTableDM[iRow,iCol] != null) {
    PlotTableDM[iRow,iCol].ShowLegend = true; //iCol == 0 && iRow == 0;
    PlotTableDM[iRow,iCol].LegendFont = 10;
    //if(iRow == 0)
    //   PlotTable[iRow,iCol].Title = aPis[iCol];
    if(iCol == 0)
       PlotTableDM[iRow,iCol].Ylabel = caseS[iRow];
}
}
}
}
}

J432k3_amr1_case1_Oh0.1_AnalytInit_Newton
J432k3_amr1_case1_Oh0.1_AnalytInit
J432k3_amr1_case2_Oh0.1_AnalytInit_Newton
J432k3_amr1_case2_Oh0.1_AnalytInit
J432k3_amr1_case3_Oh0.1_AnalytInit_Newton
J432k3_amr1_case3_Oh0.1_AnalytInit
J432k3_amr1_case4_Oh0.1_AnalytInit_Newton
J432k3_amr1_case4_Oh0.1_AnalytInit
J432k3_amr1_case5_Oh0.56_AnalytInit_Newton
J432k3_amr1_case5_Oh0.56_AnalytInit


# Case 1

In [ ]:
PlotTableDM[0,0].XrangeMin = 0;
PlotTableDM[0,0].XrangeMax = 1;
PlotTableDM[0,0].Legend_maxrows = 4;
PlotTableDM[0,0].LegendAlignment = new [] { "i", "t", "c" };
var gp = PlotTableDM[0,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",10" inside top center  maxrows 4 Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.7 
 
 
 
 
 0.8 
 
 
 
 
 0.9 
 
 
 
 
 1 
 
 
 
 
 1.1 
 
 
 
 
 1.2 
 
 
 
 
 1.3 
 
 
 
 
 1.4 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 case1 
 
 
 
 
 theta0-Ainit 
 
 
 theta0-Ainit 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M344.4,52.6 L381.0,52.6 M71.9,64.1 L75.3,99.4 L78.7,95.6 L82.1,94.4 L85.5,89.7 L88.9,87.1
 L92.2,85.9 L95.6,85.6 L99.0,86.0 L102.4,86.8 L105.8,88.1 L109.2,89.7 L112.6,91.7 L116.0,94.1
 L119.4,96.7 L122.8,99.6 L126.2,102.8 L129.5,106.3 L132.9,110.0 L136.3,113.8 L139.7,117.9 L143.1,122.1
 L146.5,126.5 L149.9,131.0 L153.3,135.5 L156.7,140.2 L160.1,144.8 L163.5,149.5 L166.8,154.2 L170.2,158.9
 L173.6,163.5 L177.0,168.4 L180.4,173.2 L183.8,178.0 L187.2,182.7 L190.6,187.5 L194.0,192.1 L197.4,195.4
 L200.8,199.5 L204.1,203.5 L207.5,207.4 L210.9,211.1 L214.3,214.6 L217.7,218.0 L221.1,221.2 L224.5,224.3
 L227.9,227.2 L231.3,229.9 L234.7,232.6 L238.1,236.8 L241.5,239.1 L244.8,241.3 L248.2,243.4 L251.6,245.3
 L255.0,247.2 L258.4,249.0 L261.8,250.7 L265.2,252.3 L268.6,253.9 L272.0,255.5 L275.4,257.1 L278.8,262.8
 L282.1,261.5 L285.5,261.5 L288.9,262.4 L292.3,263.6 L295.7,264.9 L299.1,266.3 L302.5,267.6 L305.9,268.9
 L309.3,270.2 L312.7,271.5 L316.1,272.7 L319.4,274.0 L322.8,275.2 L326.2,276.4 L329.6,277.5 L333.0,278.7
 L336.4,279.8 L339.8,281.0 L343.2,282.1 L346.6,283.3 L350.0,286.6 L353.4,287.6 L356.7,288.7 L360.1,289.8
 L363.5,290.8 L366.9,291.9 L370.3,293.0 L373.7,294.2 L377.1,295.4 L380.5,296.9 L383.9,299.0 L387.3,299.6
 L390.7,300.8 L394.0,302.2 L397.4,303.8 L400.8,305.4 L404.2,310.7 L407.6,312.0 L411.0,310.6 L414.4,312.4
 L417.8,314.3 L421.2,316.2 L424.6,318.2 L428.0,321.3 L431.3,323.1 L434.7,325.0 L438.1,327.1 L441.5,329.1
 L444.9,331.1 L448.3,333.3 L451.7,335.6 L455.1,338.0 L458.5,340.6 L461.9,343.3 L465.3,346.1 L468.6,348.9
 L472.0,354.9 L475.4,358.2 L478.8,360.4 L482.2,362.8 L485.6,367.7 L489.0,368.2 L492.4,371.0 L495.8,374.0
 L499.2,377.9 L502.6,380.9 L505.9,384.0 L509.3,387.2 L512.7,390.4 L516.1,393.7 L519.5,397.0 L522.9,402.0
 L526.3,403.7 L529.7,407.1 L533.1,410.4 L536.5,413.8 L539.9,417.2 L543.2,420.6 L546.6,423.9 L550.0,427.3
 L553.4,430.6 L556.8,433.9 L560.2,437.1 L563.6,440.3 L567.0,443.5 L570.4,446.6 L573.8,449.8 L577.2,452.9
 L580.6,456.0 L583.9,459.0 L587.3,462.0 L590.7,465.0 L594.1,467.9 L597.5,470.8 L600.9,473.5 L604.3,476.3
 L607.7,478.9 L611.1,481.5 L614.5,484.0 L617.9,486.5 L621.2,488.9 L624.6,491.2 L628.0,493.4 L631.4,495.6
 L634.8,497.7 L638.2,499.7 L641.6,501.7 L645.0,503.6 L648.4,505.4 L651.8,507.2 L655.2,508.8 '/> 
 
 theta90x-Ainit 
 
 
 theta90x-Ainit 
 
 
 
	<path stroke='rgb( 0, 255, 0)' d='M344.4,67.6 L381.0,67.6 M71.9,521.5 L75.3,522.1 L78.7,522.2 L82.1,522.2 L85.5,522.0 L88.9,521.5
 L92.2,520.9 L95.6,520.1 L99.0,519.2 L102.4,518.3 L105.8,517.2 L109.2,516.1 L112.6,514.9 L116.0,513.7
 L119.4,512.4 L122.8,511.2 L126.2,509.9 L129.5,508.5 L132.9,507.2 L136.3,505.9 L139.7,504.5 L143.1,503.2
 L146.5,501.8 L149.9,500.5 L153.3,499.1 L156.7,497.7 L160.1,496.2 L163.5,494.8 L166.8,493.3 L170.2,491.7
 L173.6,490.2 L177.0,488.6 L180.4,487.0 L183.8,485.3 L187.2,483.6 L190.6,481.8 L194.0,480.0 L197.4,478.1
 L200.8,476.2 L204.1,474.2 L207.5,472.1 L210.9,470.0 L214.3,467.8 L217.7,465.6 L221.1,463.3 L224.5,460.9
 L227.9,458.5 L231.3,456.0 L234.7,453.5 L238.1,450.8 L241.5,448.2 L244.8,445.5 L248.2,442.7 L251.6,439.9
 L255.0,437.0 L258.4,434.2 L261.8,431.3 L265.2,428.3 L268.6,425.4 L272.0,422.4 L275.4,419.4 L278.8,414.9
 L282.1,411.9 L285.5,409.4 L288.9,406.9 L292.3,404.2 L295.7,401.6 L299.1,398.9 L302.5,396.3 L305.9,393.6
 L309.3,39

## Plotting energies

In [ ]:
string[] energyKeys = new string[] {"kineticEnergy", "surfaceEnergy", "totalEnergy"}; //, "kineticDissipation"};

In [ ]:
Plot2Ddata[,] PlotTableEnrgy = new Plot2Ddata[5,1];
for(int iCol = 0; iCol < 1; iCol++) {
for(int iRow = 0; iRow < 5; iRow++) { // loop over cases
for(int iAMR = 0; iAMR < 1; iAMR++) {
for(int iAna = 0; iAna < 1; iAna++) {
for(int iNew = 0; iNew < 2; iNew++) {
    string _amr = amrS[iAMR];
    string _case = caseS[iRow];

    //ISessionInfo SI = null;
    var SIs = wmg.Sessions.Where(sess => sess.Name.Contains(_case) 
                                      && sess.Name.Contains(_amr)
                                      && (sess.Name.Contains("_AnalytInit") == ShouldAnaInit[iAna])
                                      && (sess.Name.Contains("_Newton") == useNewton[iNew]));
    
    if(SIs.Count() > 1) {
        foreach(var s in SIs)
            Console.WriteLine(s);
    }
    
    ISessionInfo SI = SIs.Single();
    if(SI == null)
        continue;
    Console.WriteLine(SI.Name);

    IDictionary<string, IList<double>> tab = null;
    try {
        tab = SI.ReadTabulatedTextFileAsDoubles("EnergyLogValues.txt", '\t');
    } catch (Exception e) {
        Console.Error.WriteLine(e.Message);
        Console.WriteLine("Skipping " + SI);
        
        var dummyPlot = new Plot2Ddata();
        dummyPlot.AddDataGroup("empty", new double[]{ 0, 1 }, new double[] { 0, 1 });
        PlotTableDM[iRow,iCol] = dummyPlot;
        continue;
    }
    
    var plot = new Plot2Ddata();
    var allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
    var time = tab["time"];
    int cnt = -1;
    foreach(var key in energyKeys) {
       cnt++;
       var fmt = new PlotFormat();
       fmt.Style = Styles.Lines; 
       fmt.LineColor = allColors[cnt%allColors.Length];
        
        // if(iAMR == 1 && ShouldAnaInit[iAna] == false)
        //     fmt.DashType = DashTypes.Solid;
        // else if(iAMR == 1 && ShouldAnaInit[iAna] == true)
        //     fmt.DashType = DashTypes.Dashed;
        // else if(iAMR == 0 && ShouldAnaInit[iAna] == false)
        //     fmt.DashType = DashTypes.Dotted;
        // else if(iAMR == 0 && ShouldAnaInit[iAna] == true)
        //     fmt.DashType = DashTypes.DotDotDashed;
        // else
        //     throw new Exception("unknown setting");
        if (useNewton[iNew]) 
            fmt.DashType = DashTypes.Dashed;
        else  
            fmt.DashType = DashTypes.Solid;

        string name = key;
        if(iAMR > 0)
            name = name + "-amr";
        if(ShouldAnaInit[iAna])
            name = name + "-Ainit";
        if(useNewton[iNew])
            name = name + "-Newton";
        
        plot.AddDataGroup(name, time, tab[key], fmt);
   }
   
   //plot.ShowLegend = iCol == 2 && iRow == 2;
   plot.ShowLegend = true;
   
   //plot.Title = SI.Name;
    
   if(iAMR == 0) {
       PlotTableEnrgy[iRow,iCol] = plot;
   } else {
       PlotTableEnrgy[iRow,iCol] = PlotTableEnrgy[iRow,iCol].Merge(plot);
   }
   
    
   PlotTableEnrgy[iRow,iCol].XrangeMin = 0;
   PlotTableEnrgy[iRow,iCol].XrangeMax = 4;
}

if(PlotTableEnrgy[iRow,iCol] != null) {
    PlotTableEnrgy[iRow,iCol].ShowLegend = true; //iCol == 0 && iRow == 0;
    PlotTableEnrgy[iRow,iCol].LegendFont = 10;
    //if(iRow == 0)
    //   PlotTable[iRow,iCol].Title = aPis[iCol];
    if(iCol == 0)
       PlotTableEnrgy[iRow,iCol].Ylabel = caseS[iRow];
}
}
}
}
}

J432k3_amr1_case1_Oh0.1_AnalytInit_Newton
J432k3_amr1_case1_Oh0.1_AnalytInit
J432k3_amr1_case2_Oh0.1_AnalytInit_Newton
J432k3_amr1_case2_Oh0.1_AnalytInit
J432k3_amr1_case3_Oh0.1_AnalytInit_Newton
J432k3_amr1_case3_Oh0.1_AnalytInit
J432k3_amr1_case4_Oh0.1_AnalytInit_Newton
J432k3_amr1_case4_Oh0.1_AnalytInit
J432k3_amr1_case5_Oh0.56_AnalytInit_Newton
J432k3_amr1_case5_Oh0.56_AnalytInit


## Case1

In [ ]:
PlotTableEnrgy[0,0].XrangeMin = 0;
PlotTableEnrgy[0,0].XrangeMax = 1;
PlotTableEnrgy[0,0].Legend_maxrows = 4;
PlotTableEnrgy[0,0].LegendAlignment = new [] { "i", "t", "c" };
var gp = PlotTableEnrgy[0,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",10" inside top center  maxrows 4 Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 4.5 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 case1 
 
 
 
 
 kineticEnergy-Ainit 
 
 
 kineticEnergy-Ainit 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M327.1,52.6 L363.7,52.6 M71.9,422.7 L75.3,533.4 L78.7,539.5 L82.1,542.7 L85.5,544.8 L88.9,546.4
 L92.2,547.6 L95.6,548.5 L99.0,549.3 L102.4,549.8 L105.8,550.3 L109.2,550.6 L112.6,550.9 L116.0,551.1
 L119.4,551.3 L122.8,551.4 L126.2,551.5 L129.5,551.5 L132.9,551.5 L136.3,551.5 L139.7,551.4 L143.1,551.4
 L146.5,551.3 L149.9,551.2 L153.3,551.1 L156.7,551.0 L160.1,550.9 L163.5,550.8 L166.8,550.6 L170.2,550.5
 L173.6,550.4 L177.0,550.2 L180.4,550.1 L183.8,549.9 L187.2,549.8 L190.6,549.7 L194.0,549.5 L197.4,549.4
 L200.8,549.2 L204.1,549.1 L207.5,548.9 L210.9,548.8 L214.3,548.6 L217.7,548.5 L221.1,548.3 L224.5,548.2
 L227.9,548.0 L231.3,547.9 L234.7,547.7 L238.1,547.6 L241.5,547.5 L244.8,547.4 L248.2,547.3 L251.6,547.1
 L255.0,547.0 L258.4,547.0 L261.8,546.9 L265.2,546.8 L268.6,546.7 L272.0,546.6 L275.4,546.6 L278.8,546.5
 L282.1,546.5 L285.5,546.5 L288.9,546.4 L292.3,546.4 L295.7,546.4 L299.1,546.4 L302.5,546.4 L305.9,546.4
 L309.3,546.5 L312.7,546.5 L316.1,546.5 L319.4,546.6 L322.8,546.6 L326.2,546.7 L329.6,546.8 L333.0,546.9
 L336.4,546.9 L339.8,547.0 L343.2,547.1 L346.6,547.2 L350.0,547.4 L353.4,547.5 L356.7,547.6 L360.1,547.7
 L363.5,547.9 L366.9,548.0 L370.3,548.1 L373.7,548.3 L377.1,548.4 L380.5,548.6 L383.9,548.8 L387.3,548.9
 L390.7,549.1 L394.0,549.2 L397.4,549.4 L400.8,549.6 L404.2,549.8 L407.6,549.9 L411.0,550.1 L414.4,550.3
 L417.8,550.5 L421.2,550.7 L424.6,550.8 L428.0,551.0 L431.3,551.2 L434.7,551.4 L438.1,551.6 L441.5,551.8
 L444.9,552.0 L448.3,552.1 L451.7,552.3 L455.1,552.5 L458.5,552.7 L461.9,552.9 L465.3,553.1 L468.6,553.3
 L472.0,553.4 L475.4,553.6 L478.8,553.8 L482.2,554.0 L485.6,554.2 L489.0,554.4 L492.4,554.6 L495.8,554.7
 L499.2,554.9 L502.6,555.1 L505.9,555.3 L509.3,555.5 L512.7,555.7 L516.1,555.9 L519.5,556.1 L522.9,556.3
 L526.3,556.4 L529.7,556.6 L533.1,556.8 L536.5,557.0 L539.9,557.2 L543.2,557.4 L546.6,557.6 L550.0,557.8
 L553.4,558.0 L556.8,558.1 L560.2,558.2 L563.6,558.5 L567.0,558.7 L570.4,558.9 L573.8,559.1 L577.2,559.3
 L580.6,559.4 L583.9,559.6 L587.3,559.8 L590.7,560.0 L594.1,560.2 L597.5,560.3 L600.9,560.5 L604.3,560.7
 L607.7,560.8 L611.1,561.0 L614.5,561.1 L617.9,561.3 L621.2,561.4 L624.6,561.6 L628.0,561.7 L631.4,561.8
 L634.8,562.0 L638.2,562.1 L641.6,562.2 L645.0,562.3 L648.4,562.5 L651.8,562.6 L655.2,562.7 '/> 
 
 surfaceEnergy-Ainit 
 
 
 surfaceEnergy-Ainit 
 
 
 
	<path stroke='rgb( 0, 255, 0)' d='M327.1,67.6 L363.7,67.6 M71.9,174.3 L75.3,180.5 L78.7,180.6 L82.1,180.5 L85.5,180.6 L88.9,180.6
 L92.2,180.7 L95.6,180.7 L99.0,180.8 L102.4,180.9 L105.8,181.0 L109.2,181.1 L112.6,181.2 L116.0,181.3
 L119.4,181.5 L122.8,181.7 L126.2,181.9 L129.5,182.1 L132.9,182.3 L136.3,182.5 L139.7,182.8 L143.1,183.0
 L146.5,183.3 L149.9,183.6 L153.3,183.9 L156.7,184.2 L160.1,184.5 L163.5,184.8 L166.8,185.1 L170.2,185.4
 L173.6,185.7 L177.0,186.0 L180.4,186.4 L183.8,186.7 L187.2,187.0 L190.6,187.3 L194.0,187.7 L197.4,188.0
 L200.8,188.4 L204.1,188.7 L207.5,189.1 L210.9,189.4 L214.3,189.8 L217.7,190.1 L221.1,190.5 L224.5,190.8
 L227.9,191.2 L231.3,191.5 L234.7,191.8 L238.1,192.2 L241.5,192.5 L244.8,192.9 L248.2,193.2 L251.6,193.5
 L255.0,193.8 L258.4,194.1 L261.8,194.5 L265.2,194.7 L268.6,195.0 L272.0,195.3 L275.4,195.6 L278.8,196.0
 L282.1,196.2 L285.5,196.4 L288.9,1